# Incubation experiment data

This notebook takes the data collected during incubation experiments and produces interpolated respiration rate data at distinct time intervals. This data is then stored in hydroshare and can be read into the main analysis script.

## Import packages

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt

## Defining a class RespirationRate that stores values, sampling time, weight, etc.

In [13]:
class RespirationRate:
    # Constructor function of this class
    def __init__(self, rawTime, rawData, sampleID, totalMass, dryMass, waterMass, depth, note, 
                 temperature = 20, moisture = 0.6):
        # Save all the inputs in this class
        self.rawTime = rawTime
        self.rawData = rawData
        self.sampleID = sampleID
        self.totalMass = totalMass
        self.dryMass = dryMass
        self.waterMass = waterMass
        self.depth = depth
        self.temperature = temperature
        self.moisture = moisture
        self.note = note #aktaddon
        
        # Calculate respiration rate at each timestamp
        # Initialize the list. Start from respiration rate = 0 at time = 0
        self.timeElapsed = [0.] # Store time in hour
        self.respirationRate = [0.] # Store respiration rate in ugC/g-dry-soil/hour
        for i in range(len(self.rawTime)-1):
            timeHour = (self.rawTime[i+1] - self.rawTime[i]).total_seconds()/3600 # [hour]
            self.timeElapsed.append(timeHour + self.timeElapsed[-1])
            ugC = (247 - self.dryMass/2.6 - self.waterMass)*self.rawData[i+1]*0.04085/1000*12 # [ugC]
            self.respirationRate.append(ugC/self.dryMass/timeHour)
    
    # Multiply every element in respiration rate by a factor 
    def multiply(self, number):
        for i in range(len(self.respirationRate)):
            self.respirationRate[i] = self.respirationRate[i] * number
    
    # Add a value to every element in respiration rate
    def plus(self, number):
        for i in range(len(self.respirationRate)):
            self.respirationRate[i] = self.respirationRate[i] + number
    
    # This function returns respiration rate at any given time [hour]. 
    # Rate is linearly interpolated between measurements.
    # If time specified is longer than the last measurement, 
    # it's calculated as the trajectory of the last two measurements.
    def atTime(self, time):
        # If time is beyond the last measurement
        if time>self.timeElapsed[-1]:
            print('Warning: Time specified is beyond the last measurement, use trajectory.')
            # If there's only one datapoint, then no trajectory can be made. 
            if len(timeElapsed) < 2:
                raise Exception('Error making trajectory: No enough datapoints.')
            slope = (self.respirationRate[-1] - self.respirationRate[-2])/(self.timeElapsed[-1] - self.timeElapsed[-2])
            return self.respirationRate[-1] + slope*(time - self.timeElapsed[-1])
        # Else, use linear interpolation between measurements
        for i in range(len(self.timeElapsed)-1):
            if self.timeElapsed[i+1] > time:
                slope = (self.respirationRate[i+1] - self.respirationRate[i])/(self.timeElapsed[i+1] - self.timeElapsed[i])
                return self.respirationRate[i] + slope*(time - self.timeElapsed[i])
                
    def plot(self):
        plt.figure()
        plt.plot(self.timeElapsed, self.respirationRate)
        plt.xlabel('Time [hour]',fontsize=13,labelpad=10)
        plt.ylabel('Respiration rate [ugC/gSoil/hour]', fontsize=13,labelpad=10)
        plt.title('Respiration rate for study: ' + self.sampleID)
    

## Storing the data collected during the experiments

In [14]:
data = [
    # Jar 1
    RespirationRate(rawTime = [dt(2019,9,30,14,20), dt(2019,9,30,21,8), dt(2019,10,1,9,24), dt(2019,10,2,9,57), 
                               dt(2019,10,3,10,3), dt(2019,10,4,9,34), dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 929, 864, 806, 749, 631, 462, 723, 762, 757, 699],
                    sampleID = 'VMS2636351', totalMass = 281.16, dryMass = 53.35, waterMass = 7.12, depth = 15.796, note = 1),
    # Jar 2
    RespirationRate(rawTime = [dt(2019,9,30,14,40), dt(2019,9,30,21,7), dt(2019,10,1,18,31), dt(2019,10,2,10,5), 
                               dt(2019,10,3,10,8), dt(2019,10,4,9,36), dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 291, 289, 221, 294, 672, 860, 862, 740, 639, 523],
                    sampleID = 'VMS2232', totalMass = 332.21, dryMass = 103.97, waterMass = 13.52, depth = 5.744, note = 1),
    # Jar 3
    RespirationRate(rawTime = [dt(2019,9,30,14,51), dt(2019,10,1,9,20), dt(2019,10,1,18,29), dt(2019,10,2,10,0), 
                               dt(2019,10,3,10,6), dt(2019,10,4,9,38), dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 356, 83, 56, 612, 857, 647, 713, 630, 564, 499],
                    sampleID = 'VMS2175203', totalMass = 277.12, dryMass = 53.38, waterMass = 7.04, depth = 4.683, note = 1),
    # Jar 4
    RespirationRate(rawTime = [dt(2019,9,30,15,5), dt(2019,10,1,9,24), dt(2019,10,2,10,8), dt(2019,10,3,10,10), 
                               dt(2019,10,4,9,41), dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 340, 268, 519, 867, 760, 769, 662, 550, 472],
                    sampleID = 'VMS2404254', totalMass = 326.78, dryMass = 98.24, waterMass = 13.64, depth = 10.427, note = 1),
    # Jar 5
    RespirationRate(rawTime = [dt(2019,10,1,12,17), dt(2019,10,2,10,16), dt(2019,10,3,10,17), dt(2019,10,4,9,43), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 492, 627, 1022, 840, 919, 878, 867, 797],
                    sampleID = 'VMS24755', totalMass = 277.9, dryMass = 54.37, waterMass = 7.26, depth = 1.437, note = 1),
    # Jar 6
    RespirationRate(rawTime = [dt(2019,10,1,12,29), dt(2019,10,2,10,13), dt(2019,10,3,10,19), dt(2019,10,4,9,45), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 262, 261, 342, 977, 926, 674, 561, 502],
                    sampleID = 'VMS2175206', totalMass = 277.09, dryMass = 54.76, waterMass = 7.46, depth = 4.683, note = 1),
    # Jar 7
    RespirationRate(rawTime = [dt(2019,10,1,12,42), dt(2019,10,2,10,11), dt(2019,10,3,10,13), dt(2019,10,4,9,47), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 395, 302, 399, 744, 769, 610, 531, 480],
                    sampleID = 'VMS2557', totalMass = 274.97, dryMass = 53.72, waterMass = 7.3, depth = 13.736, note = 1),
    # Jar 8
    RespirationRate(rawTime = [dt(2019,10,1,12,52), dt(2019,10,2,10,58), dt(2019,10,3,10,21), dt(2019,10,4,9,49), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 1050, 878, 765, 688, 730, 729, 713, 637],
                    sampleID = 'VMS2636358', totalMass = 276.84, dryMass = 53.84, waterMass = 7.05, depth = 15.796, note = 1),
    # Jar 9
    RespirationRate(rawTime = [dt(2019,10,1,14,7), dt(2019,10,2,10,18), dt(2019,10,3,10,15), dt(2019,10,4,9,52), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 482, 277, 339, 335, 420, 423, 382, 345],
                    sampleID = 'VMS2636359', totalMass = 275.92, dryMass = 54.58, waterMass = 6.53, depth = 15.796, note = 2),
    # Jar 10
    RespirationRate(rawTime = [dt(2019,10,1,16,18), dt(2019,10,2,10,20), dt(2019,10,3,10,29), dt(2019,10,4,9,53), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 631, 384, 292, 352, 759, 691, 606, 561],
                    sampleID = 'VMS21752010', totalMass = 275.09, dryMass = 50.88, waterMass = 10.7, depth = 4.683, note = 2),
    # Jar 11
    RespirationRate(rawTime = [dt(2019,10,1,16,30), dt(2019,10,2,10,22), dt(2019,10,3,15,3), dt(2019,10,4,9,56), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 626, 385, 225, 295, 690, 637, 538, 451],
                    sampleID = 'VMS21752011', totalMass = 258.88, dryMass = 52.54, waterMass = 10.82, depth = 4.683, note = 2),
    # Jar 12
    RespirationRate(rawTime = [dt(2019,10,1,17,0), dt(2019,10,2,10,26), dt(2019,10,3,10,27), dt(2019,10,4,9,59), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 591, 541, 439, 608, 945, 891, 627, 555],
                    sampleID = 'VMS25512', totalMass = 262.74, dryMass = 55.48, waterMass = 7.26, depth = 13.736, note = 2),
    # Jar 13
    RespirationRate(rawTime = [dt(2019,10,1,18,23), dt(2019,10,2,10,28), dt(2019,10,3,10,24), dt(2019,10,4,10,1), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 702, 513, 401, 496, 829, 781, 671, 566],
                    sampleID = 'VMS24042513', totalMass = 281.22, dryMass = 58.34, waterMass = 7.32, depth = 10.427, note = 2),
    # Jar 14
    RespirationRate(rawTime = [dt(2019,10,3,15,20), dt(2019,10,3,22,15), dt(2019,10,4,10,3), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 237, 166, 172, 539, 1033, 720, 545],
                    sampleID = 'VMS2751014', totalMass = 263.78, dryMass = 54.85, waterMass = 7.15, depth = 2.186, note = 1),
    # Jar 15
    RespirationRate(rawTime = [dt(2019,10,3,15,30), dt(2019,10,3,22,17), dt(2019,10,4,10,5), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 166, 139, 183, 406, 908, 625, 476],
                    sampleID = 'VMS2283215', totalMass = 264.5, dryMass = 50.88, waterMass = 6.67, depth = 7.492, note = 1),
    # Jar 16
    RespirationRate(rawTime = [dt(2019,10,3,15,37), dt(2019,10,3,22,18), dt(2019,10,4,10,14), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 190, 152, 172, 1177, 1569, 939, 669],
                    sampleID = 'VMS21752016', totalMass = 268.31, dryMass = 56.94, waterMass = 7.44, depth = 4.683, note = 1),
    # Jar 17
    RespirationRate(rawTime = [dt(2019,10,3,15,58), dt(2019,10,3,22,19), dt(2019,10,4,10,18), 
                               dt(2019,10,5,14,0), dt(2019,10,7,15,0), 
                               dt(2019,10,9,15,0), dt(2019,10,11,15,0), dt(2019,10,13,15,0)],
                    rawData = [0, 207, 149, 179, 187, 351, 577, 522],
                    sampleID = 'VMS22317', totalMass = 264.64, dryMass = 55.1, waterMass = 7.39, depth = 5.744, note = 2),
]

## other useful functions

In [15]:
def getIndexPositions(listOfElements, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    indexPosList = []
    indexPos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            indexPos = listOfElements.index(element, indexPos)
            # Add the index position in list
            indexPosList.append(indexPos)
            indexPos += 1
        except ValueError as e:
            break
 
    return indexPosList

## Generate tables of interpolated respiration rate values

In [17]:
# Download the incubation data for table
timePlotted = [0,24,48,72,120,168,230] # (hours) time points of interest

for j in list(range(0, np.size(timePlotted))):
    tempRate = []
    tempDepth = []
    tempCol = [] #aktaddon
    for i in data:
        tempRate.append(i.atTime(timePlotted[j]))
        tempDepth.append(i.depth)
        tempCol.append(i.note) #aktaddon

    all_sample_Rate = list();
    all_sample_Depth = list()
    indexPosList = getIndexPositions(tempCol, 1)
    for i in indexPosList:
        all_sample_Rate.append(tempRate[i])
        all_sample_Depth.append(tempDepth[i])
    rock_chip_Rate = list();
    rock_chip_Depth = list()
    indexPosList = getIndexPositions(tempCol, 2)
    for i in indexPosList:
        rock_chip_Rate.append(tempRate[i])
        rock_chip_Depth.append(tempDepth[i])
    mult_coeff = 2.65*24*365

    if j ==0:
        d = {'depth_m': all_sample_Depth}
        all_sample = pd.DataFrame(data=d)
        all_sample[timePlotted[j]] = np.multiply(all_sample_Rate,mult_coeff)
        
        d = {'depth_m': rock_chip_Depth}
        rock_chip_all = pd.DataFrame(data=d)
        rock_chip_all[timePlotted[j]] = np.multiply(rock_chip_Rate,mult_coeff)
    else:
        all_sample[timePlotted[j]] = np.multiply(all_sample_Rate,mult_coeff) 
        rock_chip_all[timePlotted[j]] = np.multiply(rock_chip_Rate,mult_coeff)

all_sample.to_csv('all_sample_incubation.csv')

rock_chip_all.to_csv('rock_chip_incubation.csv')


In [25]:
indexPosList

[8, 9, 10, 11, 12, 16]

In [24]:
tempRate

[0.03193884976292464,
 0.0135177385487756,
 0.025648270933971682,
 0.01279099825677653,
 0.03575633370061076,
 0.02405871288167971,
 0.02295409234363589,
 0.029819300964098273,
 0.016069210214343498,
 0.02703218533350294,
 0.023321074911765945,
 0.02697576878951528,
 0.026093716128167084,
 0.023633121441952214,
 0.022398507267388432,
 0.02817726442678148,
 0.021550934645236976]

In [26]:
rock_chip_all

,depth_m,0,24,48,72,120,168,230
0,15.796,0.0,1002.978002,550.133633,639.929273,469.747727,397.535899,373.030646
1,4.683,0.0,1467.471461,729.513303,605.321861,684.126103,722.881658,627.525150
2,4.683,0.0,1422.738599,622.445873,541.987106,581.993946,638.546355,541.375433
3,13.736,0.0,1379.303041,960.811487,869.832686,912.514589,846.380840,626.215497
4,10.427,0.0,1538.177628,844.454111,728.933687,731.277715,701.245990,605.739526
5,5.744,0.0,505.769509,286.705997,228.250371,254.179036,440.327181,500.283397


In [18]:
all_sample

,depth_m,0,24,48,72,120,168,230
0,15.796,0.0,2942.039223,1521.231290,1417.969056,759.784440,691.333630,741.428458
1,5.744,0.0,406.436875,292.698015,326.029505,637.637890,374.381463,313.800783
2,4.683,0.0,613.591265,373.902615,1291.951236,1060.227645,681.846233,595.398961
3,10.427,0.0,380.292847,295.674587,561.553839,602.441525,355.457751,296.930234
4,1.437,0.0,1039.367077,1264.132430,1939.660741,1132.291332,849.576765,830.047531
5,4.683,0.0,542.090324,507.494351,750.423866,1243.387775,759.311240,558.498961
6,13.736,0.0,824.826341,604.383785,830.824918,993.990815,665.553341,532.856300
7,15.796,0.0,2164.092537,1715.332971,1469.550909,925.469045,696.462228,692.225253
8,2.186,0.0,570.559755,285.270583,392.995138,741.277853,826.674805,548.619281
9,7.492,0.0,531.558438,326.272999,368.121744,677.012452,785.408346,519.958948
